## Task ba9h

In [78]:
def read_data(fname='rosalind_ba9h.txt'):
    p_list = []
    with open(fname, 'r') as f:
        s = f.readline().strip()
        while 1:
            s2 = f.readline()
            if s2 != '':
                p_list.append(s2.strip())
            else:
                break
    return s, p_list

In [10]:
def get_list_of_suffixes(s):
    # return suffix as well as its index and 2 placeholders for 2 rank variables
    return [[s[i:], i, None, None] for i in range(len(s))]

In [19]:
def get_suffix_array(s):
    n = len(s)
    # nlog^2n algorithm
    suf = get_list_of_suffixes(s)
    
    #2 character sort
    for i in range(n):
        suf[i][2] = ord(s[i]) - ord("a") # ranks for 1st character
        suf[i][3] = ord(s[i+1]) - ord("a") if i + 1 < n else -1
    suf = sorted(suf, key = lambda x: (x[2], x[3]))

    #k = 2^i character sort (i >=2)
    idxs = [0] * n
    k = 4
    # first suffix
    while k < n * 2:
        cur_rank = 0
        prev_rank = suf[0][2]
        suf[0][2] = cur_rank
        idxs[suf[0][1]] = 0
        
        #rest
        for i in range(1, n):
            # compare by pair of ranks
            if suf[i][2] == prev_rank and suf[i][3] == suf[i-1][3]:
                prev_rank = suf[i][2]
                suf[i][2] = cur_rank
            
            else:
                prev_rank = suf[i][2]
                cur_rank += 1
                suf[i][2] = cur_rank
            
            idxs[suf[i][1]] = i
            
        # fill 2nd rank var for each suffix
        for i in range(n):
            next_idx = suf[i][1] + k // 2
            suf[i][3] = suf[idxs[next_idx]][2] if next_idx < n else -1
            
        # sort by rank pair again
        suf = sorted(suf, key = lambda x: (x[2], x[3]))
        k *= 2
        
    suf_arr = [elem[1] for elem in suf]
    return suf_arr

In [130]:
def search_pattern(s, p, suf_arr=None):
    if suf_arr is None:
        suf_arr = get_suffix_array(s)
    n = len(s)
    m = len(p)
#     print(suf_arr)
    suffs = get_list_of_suffixes(s)
    # bin search for pattern in suffix arr
    l, r = 0, n-1
    while l <= r:
        mid = l + (r - l) // 2
        mid_suf = s[suf_arr[mid]:suf_arr[mid]+m]
        if mid_suf == p:
            l2 = l
            r2 = r
            # tighten the range of l and r since they may also match
            while l2 <= mid:
                l2_suf = s[suf_arr[l2]:suf_arr[l2]+m]
                if l2_suf == p:
                    break
                l2 += 1
            while r2 >= mid:
                r2_suf = s[suf_arr[r2]:suf_arr[r2]+m]
                if r2_suf == p:
                    break
                r2 -= 1
            return [suf_arr[j] for j in range(l2,r2+1)]
                
        if p < mid_suf:
            r = mid - 1
        else:
            l = mid + 1
    return -1

In [138]:
def search_patterns(s, p_list):
    suf_arr = get_suffix_array(s)
    return sorted(list(set([elem for p in p_list for elem in search_pattern(s, p, suf_arr)])))

In [142]:
def main(fname='rosalind_ba9h.txt'):
    s, p_list = read_data(fname)
    res = search_patterns(s, p_list)
    print(*res, sep=' ')

In [143]:
main('sample_data/ba9h/sample.txt')

1 4 11 15


In [145]:
# driver code
# main()